In [341]:
import pandas as pd
import re

from maple.types import Action, ActionType
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')


In [342]:
legislators = [
"Gomez",
"Cronin",
"Gobi",
"Crighton",
"Tarr",
"Stone", 
"Creem",
"Lesser",
"Chandler",
"Boncore",
"Eldridge",
"Lewis",
"Lovely",
"Comerford",
"Keenan",
"Edwards",
"Spilka",
"Brady",
"Montigny",
"Moore",
"Pacheco",
"Rodrigues",
"Rush",
"O'Connor",
"Jehlen",
"Rausch",
"Fattman",
"Chang-Diaz",
"DiDomenico",
"Moran",
"Brownsberger",
"Timilty",
"Consalvo",
"Duffy",
"Giannino",
"Kushmerek",
"Pease",
"Philips",
"Scanlon",
"Xiarhos",
"Oliveira",
"Cabral",
"D'Emilia",
"Madaro",
"Ferrante",
"Michlewitz",
"Peisch",
"Puppolo",
"Silvia",
"Ashe",
"Ayers",
"Jones",
"Murray",
"González",
"Doherty",
"Barber",
"Cronin",
"Fiola",
"LeBoeuf",
"Cahill",
"Donahue",
"Garlick",
"Gregoire",
"Linsky",
"Tucker",
"Balser",
"Holmes",
"Mariano",
"Dooley",
"Garballey",
"Harrington",
"Howitt",
"Peake",
"Pignatelli",
"Ultrino",
"Whipps",
"Farley-Bouvier",
"Golden",
"Nguyen",
"Stanley",
"Walsh",
"Whelan",
"Galvin",
"Straus",
"Turco",
"Fernandes",
"Cassidy",
"Kane",
"Hawkins",
"Arciero",
"Wagner",
"Kelcourse",
"McGonagle",
"McKenna",
"O'Day",
"Parisella",
"Roy",
"Zlotnik",
"Lipper-Garabedian",
"Ferguson",
"Gordon",
"Hogan",
"Campbell",
"Ehrlich",
"Sabadosa",
"Malia",
"Mirra",
"Devers",
"Robinson",
"Cusack",
"Day",
"Decker",
"DuBois",
"Finn",
"Keefe",
"Muratore",
"Blais",
"Higgins",
"Boldyga",
"Donato",
"Durant",
"Haddad",
"Mark",
"Schmid",
"Frost",
"Belsito",
"Hinds",
"Finegold",
"Friedman",
"DiZoglio",
"Kennedy",
"Velis",
"Cyr",
"Barrett",
"Collins",
"Feeney",
"Diggs",
"Oakley",
"Ramos",
"Sullivan",
"Vargas",
"Williams",
"Hendricks",
"Minicucci",
"Sena",
"Berthiaume",
"Robertson",
"Rogers",
"Mom",
"Gouveia",
"Moran",
"Lewis"
]


hash_reg = re.compile('\(#.*\)')

def normalizeLegislatorNames(action: str):
  matches = [l for l in legislators if l in action]
  
  for m in matches: 
    r = '\(.*' + m + '.*\)'
    action = "(###)".join(re.compile(r).split(action))
  action = "(leg_names)".join(hash_reg.split(action))
  return action


In [343]:
first_committees  = [
  "advanced information technology, the internet and cybersecurity",
  "bonding, capital expenditures and state assets", "cannabis policy" ,
  "children, families and persons with disabilities",
  "community development and small businesses",
  "consumer protection and professional licensure",
  "covid-19 and emergency preparedness and management",
  "economic development and emerging technologies",
  "higher education",
  "education",
  "elder affairs",
  "election laws",
  "environment, natural resources and agriculture",
  "export development",
  "financial services",
  "Housing and Food Security", 
  "Housing and Local Aid Social Services and Veteran Services and Soldiers",
  "Housing Energy and Environmental Affairs and com_name"
  "housing",
  "the judiciary",
  "labor and workforce development",
  "mental health substance use and recovery",
  "municipalities and regional government",
  "public health",
  "public safety and homeland security",
  "public service",
  "racial equity, civil rights and inclusion ",
  "revenue",
  "state administration and regulatory oversight",
  "telecommunications, utilities and energy",
  "tourism, arts and cultural development",
  "transportation",
  "Veterans and Federal Affairs",
  "House Bills in Third",
  "Election Laws",
  "Transportation",
  ]


chamber_committees = [
  "committee on rules",
  'committee on rules of the two branches, acting concurrently',
  "ways and means", 
  "steering, policy and scheduling",
  "steering and policy",
  "housing and finance",
  "health care financing",
  "Bills in the third reading"
]


ts = """3061,Discharged to the committee on Labor and Workforce Development,Labor and Workforce Development,uncategorized,,
199,Discharged to the committee on Mental Health Substance Use and Recovery,Mental Health Substance Use and Recovery,uncategorized,,
17682,Discharged to the committee on Municipalities and Regional Government,Municipalities and Regional Government,uncategorized,,
11091,Discharged to the committee on Public Health,Public Health,uncategorized,,
6369,Discharged to the committee on Public Safety and Homeland Security,Public Safety and Homeland Security,uncategorized,,"""


def extract_committees(action):
  found_committees = []
  for f in first_committees:
    found_committees.extend(re.compile('(' + f + ')', re.I).findall(action))
  for c in chamber_committees:
    found_committees.extend(re.compile('(' + c + ')', re.I).findall(action))
  if len(found_committees) > 0: 
    return found_committees 


def extract_first_committees(action):
  found_committees = []
  for f in first_committees:
      m = re.compile('((Joint|House|Senate|Temporary )?committee on ' + f + ')', re.I).search(action)
      if m != None: 
        found_committees.append(m.group())
  if len(found_committees) > 0: 
    return found_committees
 
def extract_chamber_committees(action):
  found_committees = []
  for c in chamber_committees:
    m = re.compile('((Joint|House|Senate|Temporary )?committee on ' + c + ')', re.I).search(action)
    if m != None: 
      found_committees.append(m.group())
  if len(found_committees) > 0: 
    return tuple(found_committees)

def replace_committees(action):
  found_committees = []
  for f in first_committees:
    found_committees.extend(re.compile(f, re.I).findall(action))
  for c in chamber_committees:
    found_committees.extend(re.compile(c, re.I).findall(action))
    
  if found_committees != None: 
    for comm in found_committees:
      action ="com_name".join(action.split(comm))
      
  return action


In [344]:
day = ['sunday', 'monday', 'tuesday','wednesday', 'thursday', 'friday', 'saturday']
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

def date_dates(action):
  for d in day: 
    dreg = re.compile(d, re.I)
    day_split = dreg.split(action)
    action = "[day]".join(day_split)
    
  for m in month: 
    mon_reg = re.compile(m, re.I)
    mon_split = mon_reg.split(action)
    action = "[month]".join(mon_split)
  
  date_reg = re.compile('\d\d/\d\d/\d\d\d\d', re.I)
  action = date_reg.sub("[date]", action)
  
  year_reg = re.compile('202\d', re.I)
  action = year_reg.sub("[year]", action)
  
  time_reg = re.compile('\d\d:\d\d(?:\s?[AP]M)?', re.I)
  action = re.sub(time_reg, "[time]", action)

  
  return action

In [345]:
def bill_number_replace(action):
  reg = re.compile('(?:[HS]\d+)', re.I)
  a = reg.findall(action)
  if len(a) > 0: 
    breg = re.compile("(.*) ".join(a), re.I)
    action = breg.sub('[billnum]', action)
  return action

def number_replace(action):
  reg = re.compile('(?:\d+,? ?)+(?:and )?', re.I)
  action = re.sub(reg, '[num]', action)
  return action

def reletter_attachment(action): 
  reg = re.compile('attachment\s\w$', re.I)
  return re.sub(reg, 'Attachment X', action)

def reletter_amendment(action): 
  reg = re.compile('amendment\s\w\W', re.I)
  return re.sub(reg, 'amendment X ', action)

def renumber_amendment(action):
  find=re.compile('Amendment', re.I)
  replace = re.compile('#?\d+\.?\d*')
  if find.search(action):
    action = "[num]".join(replace.split(action))
  return action

def comma_spacer(action):
  reg = re.compile(',\s?', re.I)
  return re.sub(reg, ', ', action)

def consolidated_amendment(action):
  consolidated_amendment = re.compile('consolidated amendment', re.I)
  replace_inside_parens = re.compile('\([^S].*\)')
  if consolidated_amendment.match(action) != None:
    action = "([com_name])".join(replace_inside_parens.split(action))
  return action  
  
def replace_vote(action):
  find = re.compile('YEAS.*NAYS', re.I)
  reg = re.compile('[\s-]*\d*\s?\(?YEAS to \d+\s?NAYS\)?\s?\(.*\)', re.I)
  if find.search(action) != None: 
      splits = reg.split(action)
      action = " [vote]".join(splits)
  return action


def norm_spellings(action):
  words = [
    ('therefor.', 'therefor'), 
    ("in the place", "in place"), 
    ("  ", " "), 
    ('committee on conference', 'committee of conference'), 
    ('\s?-\s?', ' ') ]
  for a in words: 
    action = re.sub(a[0], a[1], action, re.I )
  return action 

def remove_surround_quotes(action):
  reg = re.compile('"', re.I)
  quote = reg.search(action)
  if quote != None: 
    action = reg.sub('', action)
  return action 

def amendments_considered(action):
  reg = re.compile('^(The following amendments were considered as one and \w*)', re.I)
  capture = reg.search(action)
  if capture != None: 
      action = capture.group() + ' [amdmnts]'
  return action


def passed_to_be_engrossed(action):
  p = re.compile('(Passed to be engrossed)', re.I)
  rc = re.compile('(see Roll Call)', re.I)
  rcnum = re.compile('see Roll Call\s?(#?\d+)', re.I)
  v = re.compile('YEAS?.*NAYS?', re.I)
  pre = re.compile('(.*)Passed to be engrossed', re.I)
  
  s = p.search(action)
  sr = rc.search(action)
  num = rcnum.search(action)
  vo = v.search(action)
  prep = pre.search(action)
  
  
  if s != None: 
    if prep != None: 
      action = prep.group(1) 
    action += s.group()
    if sr != None: 
      action += " " + sr.group()
    if num != None: 
      action += " " + num.group(1)
    if vo != None: 
      action += ' [vote]'
  
  return action




In [346]:
df = pd.read_csv("predictions.csv")

df['clean_actions'] = df['action']
df['clean_actions'] = df['clean_actions'].apply(lambda s: bill_number_replace(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: normalizeLegislatorNames(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: comma_spacer(s))
df['first_committees'] = df['clean_actions'].apply(lambda row: extract_first_committees(row)) 
df['chamber_committees'] = df['clean_actions'].apply(lambda row: extract_chamber_committees(row)) 
df['clean_actions'] = df['clean_actions'].apply(lambda row: replace_committees(row)) 
df['clean_actions'] = df['clean_actions'].apply(lambda s: date_dates(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: reletter_attachment(s))

df['clean_actions'] = df['clean_actions'].apply(lambda s: replace_vote(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: reletter_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: renumber_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: amendments_considered(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: consolidated_amendment(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: norm_spellings(s))
df['clean_actions'] = df['clean_actions'].apply(lambda s: passed_to_be_engrossed(s))




In [347]:
pd.options.display.max_colwidth = -1


drp = df.drop_duplicates(subset='clean_actions') #.sort_values(by='clean_actions')


display(drp.count())



action_id             929
action                929
prediction            929
label                 54 
clean_actions         929
first_committees      13 
chamber_committees    11 
dtype: int64

In [348]:
labels= [
  'ho.se', 
  'senate',
  'committee(?: o[nf] conference)?',
  'com_name',
  'study order', 
  'discharged', 
  'hearing',
  'schedule',
  'postpone[d]?',
  'reschedule',
  'cancel',
  'placed on file',
  'calendar',
  
  'recommitted',
  
  'accompan\w*(?:\sby)?',
  
  'refer\w*(?: to)?(?: from)?',
  'recommend\w*',
  
  'bill reported favorably', 
  'bill reported adversely', 
  'report accepted', 
  'reported to', 
  'reported from'
  
  'substitut\w*',  
  'substituted for',
  
  'appointed',
  
  'striking out(?: all)?',
  'pending',
  '(?:published as )?(?:printed as )?amend\w*(?: accepted)?(?: considered)?', 
  'adopt\w*', 
  'concur\w*',
  
  'new draft',
  'new text', 
  'reprint\w*(?: as amended)?'
  
  'see',
  'see [billnum]',
  'See',
  
  'reported',
  'ought to pass',
  'ought not to pass', 
  'ought to be adopted', 
  'ought not to be adopted', 
  'ought to be approved', 
  'ought not to be approved', 
  
  
  'order\w*(?: of the day)?(?: to a third reading)?',
  
  'vote', 
  'Roll Call',
  'passed over veto',
  'house ways and means',
  'bills in the third reading',
  'steering, policy and scheduling',
  'Bonding, Capital Expenditures and Assets',
  'committee on rules of the two branches, acting concurrently',
  'committee on rules',
  
  'leg_names',
  
  'rejected',
  'laid aside',
  
  'enacted', 
  'laid before the governor',
  
  'attachment',
  'governor',
  'signed',
  'in part',
  'chapter',
  'message',
  'motion', 
  'joint session',
  'consolidated',
  'engrossed',
  
  
  '^read$',
  'read second',
  'read third',
  'first reading',
  'second reading',
  'suspended',
  'rules suspended',
  'point of order',
  'negatived',
  'returned',
  
  'Transmitted to the Secretary of State',
  'General Appropriation Bill',
  'emergency preamble',
  'constitutional amendment',
  
  'Taken out of the notice section',
  'also based on',
  'read for its information',
  'duplicate'
  
  ]



def set_labels(action):
  clean_labels = pd.Series(data="", index = range(0))
  for l in labels:
    reg = re.compile(l, re.I)
    m = reg.search(action)
    if m != None:
      lab = m.group().lower()
      ps = pd.Series(lab)
      clean_labels = clean_labels.append(ps)
  return clean_labels.array



drp['label'] = drp['clean_actions'].apply(lambda s: set_labels(s))




In [349]:
action_types = [
  (['new draft'], 'bill redirect'),
  (['new text'], 'bill redirect'),
  (['study order'], 'bill redirect'), 
  (['accompanied'], 'bill redirect'),
  (['accompanied by'], 'bill redirect'),
  (['bill reported favorably'],'committee report'),
  (['committee reported favorably'],'committee report'),
  (['bill reported unfavorably'],'committee report'),
  (['committee reported unfavorably'],'committee report'),
  (['committee reported'], 'committee report'),
  (['reported'], 'committee report'),
  (['ought to pass'], 'committee report'),
  (['ought not to pass'], 'committee report'),
  (['ought to be adopted'], 'committee report'),
  (['committee recommended'], 'committee report'),
  (['referred to the committee'], 'referred to committee'), 
  (['recommending'], 'committee report'),
  
  
  
  (['second reading'], 'status: chamber'),
  (['third reading'], 'status: chamber'),
  (['ordered to a second reading'], 'status: chamber'),
  (['ordered to a third reading'], 'status: chamber'),
  (['orders of the day'], 'status: chamber'),
  (['read'], 'read'),
  
  (['amendment'], 'amendment'),
  (['amendments'], 'amendment'),
  (['amended'], 'amended'),
  (['laid aside'], 'laid aside'),
  (['rejected'], 'rejected'),
  (['accepted'], 'accepted'),
  (['adopted'],  'adopted'),
  (['striking out'], 'striking out'),
  
  (['engrossed'], 'engrossment'),
  (['enacted'], 'enactment'),
  (['laid before the governor'], 'laid before the governor'),
  (['enacted and laid before the governor'], 'enactedment'),
  (['enacted and laid before the governor'], 'laid before the governor'),
  (['message'], 'message'),


  (['committee', 'referred'], 'referred to committee'),
  (['committee', 'referred to'], 'referred to committee'),
  (['committee', 'referred', 'bills in the third reading'], 'to bills in the third reading'),
  (['committee', 'referred to', 'bills in the third reading'], 'to bills in the third reading'),
  (['committee', 'ought to pass'], 'committee report'),
  (['committee', 'ought not to pass'], 'committee report'),
  (['committee', 'reported favorably'], 'committee report'),
  (['committee', 'reported unfavorably'], 'committee report'),
  (['calendar'], 'scheduling'),
  (['calendar','joint session'], "scheduling"),
  (['accompanied', 'new draft', 'see'], 'rewritten'),
  (['amendment', 'rejected'], 'amendment rejected'),
  (['amendment', 'adopted'], 'amendment adopted'),
  (['amendment', 'rejected', 'vote'], 'amendment rejected by vote'),
  (['amendment', 'adopted', 'vote'], 'amendment adopted by vote'),
  (['governor', 'recommend\w*'], 'governor recommendation'),
  (['report', 'accepted'], 'report accepted'),
  
  (['taken out of the notice section'], 'uncategorized'),
  (['also based on'], 'uncategorized'),
  (['read for its information'], 'uncategorized'),
  (['read'], 'read'),
  (['duplicate'], 'duplicate')
]



def get_action_types(label):
  acts = pd.Series()
  for acttype in action_types: 
    if all([a.lower() in label for a in acttype[0]]):
      acts = acts.append(pd.Series(acttype[1]))
  if acts.size > 0: 
    return acts.array
  return pd.Series('uncategorized').array
      


drp['action_type'] = drp['label'].apply(lambda l: get_action_types(l))



In [350]:


def add_to_action_type(action_type, new_type: str):
  action_s = pd.Series(action_type)
  action_s = action_s.append(pd.Series(new_type))
  return action_s.array


,action_id,action,prediction,label,clean_actions,first_committees,chamber_committees,action_type
0,1,Referred to the committee on House Ways and Means,referred,"[house, committee, com_name, referred to]",Referred to the committee on House com_name,None,None,[referred to committee]
1,2,"Reported, in part, by H4000",uncategorized,"[reported, in part]","Reported, in part, by [billnum]",None,None,[committee report]
2,3,Referred to the committee on Public Service,referred,"[committee, com_name, referred to]",Referred to the committee on com_name,[committee on Public Service],None,[referred to committee]
3,4,Senate concurred,concurred,"[senate, concurred]",Senate concurred,None,None,[uncategorized]
4,5,Hearing scheduled for 07/28/2021 from 01:00 PM-04:00 PM in Virtual Hearing,hearing_scheduled,"[hearing, schedule]",Hearing scheduled for [date] from [time] [time] in Virtual Hearing,None,None,[uncategorized]
...,...,...,...,...,...,...,...,...
44968,44969,Senate Bills in the Third Reading Report (S30) accepted,uncategorized,"[senate, com_name]",Senate com_name Report ([billnum]) accepted,None,None,[uncategorized]
44969,44970,"Governor's amendment adopted, in the form recommended by Senate Bills in the Third Reading -- see Roll Call #19 (Yeas 39 to Nays 1)",uncategorized,"[senate, com_name, recommended, amendment, adopted, see, roll call, governor]","Governor's amendment adopted, in the form recommended by Senate com_name see Roll Call [num] (Yeas [num] to Nays [num])",None,None,"[amendment, adopted, amendment adopted]"
44972,44973,"Governor's amendment adopted, in concurrence - 146 YEAS to 13 NAYS (See YEA and NAY No. 14)",uncategorized,"[amendment, adopted, concurrence, vote, governor]","Governor's amendment adopted, in concurrence [vote]",None,None,"[amendment, adopted, amendment adopted, amendment adopted by vote]"
44973,44974,Re-enacted - 145 YEAS to 14 NAYS (See YEA and NAY No. 15),uncategorized,"[vote, enacted]",Re enacted [vote],None,None,[enactment]


In [356]:

output = drp[['action_id', 'action', 'prediction', 'label', 'action_type']]


output['action_type'] = output['action_type'].apply(lambda x: pd.Series(x).to_list())
output['label'] = output['label'].apply(lambda x: pd.Series(x).to_list())

output.to_csv('./predictions_by_label.csv')

